In [1]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install pyarrow

Note: you may need to restart the kernel to use updated packages.


In [3]:
import io
import pandas as pd
from minio import Minio

# -------------------------------
# MinIO connection (matches your working DAG style)
# -------------------------------
client = Minio(
    "minio:9000",
    access_key="minioadmin",
    secret_key="minioadmin",
    secure=False,
)

BUCKET = "curated"
PREFIX = "tabular/market_ohlcv_daily/exchange=ASX/"   # narrow scope as you did

# -------------------------------
# List parquet objects
# -------------------------------
parquet_keys = [
    obj.object_name
    for obj in client.list_objects(BUCKET, prefix=PREFIX, recursive=True)
    if obj.object_name.endswith(".parquet")
]

print(f"Found {len(parquet_keys)} parquet files under s3://{BUCKET}/{PREFIX}")
print("Sample keys:", parquet_keys[:5])

# -------------------------------
# Read each parquet and concatenate
# (force trade_date to a consistent type to avoid Arrow merge errors)
# -------------------------------
dfs = []
for key in parquet_keys:
    resp = client.get_object(BUCKET, key)
    try:
        data = resp.read()  # bytes
    finally:
        resp.close()
        resp.release_conn()

    part = pd.read_parquet(io.BytesIO(data), engine="pyarrow")

    # Normalise problematic columns if present
    if "trade_date" in part.columns:
        part["trade_date"] = part["trade_date"].astype("string")
    if "exchange" in part.columns:
        part["exchange"] = part["exchange"].astype("string")

    dfs.append(part)

df = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

# -------------------------------
# Basic EDA / sanity checks
# -------------------------------
print("\n=== SHAPE ===")
print(df.shape)

print("\n=== DTYPES ===")
print(df.dtypes)

print("\n=== INFO ===")
df.info()

print("\n=== HEAD ===")
display(df.head())

print("\n=== NULL COUNTS ===")
print(df.isna().sum())

print("\n=== NULL PERCENTAGE ===")
print((df.isna().mean() * 100).round(2))


Found 1265 parquet files under s3://curated/tabular/market_ohlcv_daily/exchange=ASX/
Sample keys: ['tabular/market_ohlcv_daily/exchange=ASX/trade_date=2020-12-16/snapshot.parquet', 'tabular/market_ohlcv_daily/exchange=ASX/trade_date=2020-12-17/snapshot.parquet', 'tabular/market_ohlcv_daily/exchange=ASX/trade_date=2020-12-18/snapshot.parquet', 'tabular/market_ohlcv_daily/exchange=ASX/trade_date=2020-12-21/snapshot.parquet', 'tabular/market_ohlcv_daily/exchange=ASX/trade_date=2020-12-22/snapshot.parquet']

=== SHAPE ===
(122987, 16)

=== DTYPES ===
dataset_id                        object
exchange                  string[python]
ticker                            object
vendor_symbol                     object
currency                          object
trade_date                string[python]
open                             float64
high                             float64
low                              float64
close                            float64
volume                             in

,dataset_id,exchange,ticker,vendor_symbol,currency,trade_date,open,high,low,close,volume,adj_close,ingest_ts,ingest_batch_id,source_object_key,row_hash
0,market_ohlcv_daily,ASX,BHP,BHP.AX,AUD,2020-12-16,37.842327,38.153973,37.762192,37.886848,8056476,26.258919,2025-12-15 06:47:10.130119+00:00,scheduled__2025-12-15T06:40:00+00:00,tabular/market_ohlcv_daily/exchange=ASX/trade_...,aaf26ef976e0513f0f938f2ff17a79ccdba4a528ddc42c...
1,market_ohlcv_daily,ASX,WES,WES.AX,AUD,2020-12-16,50.970001,51.560001,50.860001,51.090000,1807871,42.098888,2025-12-15 06:47:10.130119+00:00,scheduled__2025-12-15T06:40:00+00:00,tabular/market_ohlcv_daily/exchange=ASX/trade_...,59761a7c8ecde97097f5c8a1d5299f8c6fb44fd185b98f...
2,market_ohlcv_daily,ASX,CBA,CBA.AX,AUD,2020-12-16,83.129997,84.680000,83.129997,83.849998,2037896,69.613434,2025-12-15 06:47:10.130119+00:00,scheduled__2025-12-15T06:40:00+00:00,tabular/market_ohlcv_daily/exchange=ASX/trade_...,e8f5dbbd135334b35c5c46fb26bb5791154eb4628a73d8...
3,market_ohlcv_daily,ASX,CSL,CSL.AX,AUD,2020-12-16,293.799988,293.989990,286.950012,286.950012,732842,268.164001,2025-12-15 06:47:10.130119+00:00,scheduled__2025-12-15T06:40:00+00:00,tabular/market_ohlcv_daily/exchange=ASX/trade_...,f26901219ae3f6717740f58de6288358e03050ef1559be...
4,market_ohlcv_daily,ASX,RIO,RIO.AX,AUD,2020-12-16,114.250000,114.989998,113.839996,114.370003,1212840,80.232597,2025-12-15 06:47:10.130119+00:00,scheduled__2025-12-15T06:40:00+00:00,tabular/market_ohlcv_daily/exchange=ASX/trade_...,0112148e7c9297fe14506b9703c4455d02060763b7d11b...



=== NULL COUNTS ===
dataset_id              0
exchange                0
ticker                  0
vendor_symbol           0
currency                0
trade_date              0
open                    0
high                    0
low                     0
close                   0
volume                  0
adj_close            2454
ingest_ts               0
ingest_batch_id         0
source_object_key       0
row_hash                0
dtype: int64

=== NULL PERCENTAGE ===
dataset_id           0.0
exchange             0.0
ticker               0.0
vendor_symbol        0.0
currency             0.0
trade_date           0.0
open                 0.0
high                 0.0
low                  0.0
close                0.0
volume               0.0
adj_close            2.0
ingest_ts            0.0
ingest_batch_id      0.0
source_object_key    0.0
row_hash             0.0
dtype: float64


In [4]:
bytes_used = df.memory_usage(deep=True).sum()
mb = bytes_used / (1024 ** 2)
gb = bytes_used / (1024 ** 3)

mb, gb

(np.float64(92.4315128326416), np.float64(0.09026514925062656))

In [5]:
df.memory_usage(deep=True).sort_values(ascending=False)


source_object_key    16359797
row_hash             14881427
ingest_batch_id      11437791
dataset_id            9224025
trade_date            8240129
vendor_symbol         7750707
ticker                7381746
currency              7379220
exchange              7379220
adj_close              983896
open                   983896
high                   983896
low                    983896
close                  983896
volume                 983896
ingest_ts              983896
Index                     132
dtype: int64

In [6]:
df.info(memory_usage="deep")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122987 entries, 0 to 122986
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype              
---  ------             --------------   -----              
 0   dataset_id         122987 non-null  object             
 1   exchange           122987 non-null  string             
 2   ticker             122987 non-null  object             
 3   vendor_symbol      122987 non-null  object             
 4   currency           122987 non-null  object             
 5   trade_date         122987 non-null  string             
 6   open               122987 non-null  float64            
 7   high               122987 non-null  float64            
 8   low                122987 non-null  float64            
 9   close              122987 non-null  float64            
 10  volume             122987 non-null  int64              
 11  adj_close          120533 non-null  float64            
 12  ingest_ts          122987 non-

In [7]:
bytes_per_row = df.memory_usage(deep=True).sum() / len(df)
bytes_per_row


np.float64(788.0626895525544)

In [8]:
df.shape


(122987, 16)

In [9]:

df.dtypes

dataset_id                        object
exchange                  string[python]
ticker                            object
vendor_symbol                     object
currency                          object
trade_date                string[python]
open                             float64
high                             float64
low                              float64
close                            float64
volume                             int64
adj_close                        float64
ingest_ts            datetime64[ns, UTC]
ingest_batch_id                   object
source_object_key                 object
row_hash                          object
dtype: object

In [10]:


list(df.columns)

['dataset_id',
 'exchange',
 'ticker',
 'vendor_symbol',
 'currency',
 'trade_date',
 'open',
 'high',
 'low',
 'close',
 'volume',
 'adj_close',
 'ingest_ts',
 'ingest_batch_id',
 'source_object_key',
 'row_hash']

In [11]:


df.head(3)

,dataset_id,exchange,ticker,vendor_symbol,currency,trade_date,open,high,low,close,volume,adj_close,ingest_ts,ingest_batch_id,source_object_key,row_hash
0,market_ohlcv_daily,ASX,BHP,BHP.AX,AUD,2020-12-16,37.842327,38.153973,37.762192,37.886848,8056476,26.258919,2025-12-15 06:47:10.130119+00:00,scheduled__2025-12-15T06:40:00+00:00,tabular/market_ohlcv_daily/exchange=ASX/trade_...,aaf26ef976e0513f0f938f2ff17a79ccdba4a528ddc42c...
1,market_ohlcv_daily,ASX,WES,WES.AX,AUD,2020-12-16,50.970001,51.560001,50.860001,51.090000,1807871,42.098888,2025-12-15 06:47:10.130119+00:00,scheduled__2025-12-15T06:40:00+00:00,tabular/market_ohlcv_daily/exchange=ASX/trade_...,59761a7c8ecde97097f5c8a1d5299f8c6fb44fd185b98f...
2,market_ohlcv_daily,ASX,CBA,CBA.AX,AUD,2020-12-16,83.129997,84.680000,83.129997,83.849998,2037896,69.613434,2025-12-15 06:47:10.130119+00:00,scheduled__2025-12-15T06:40:00+00:00,tabular/market_ohlcv_daily/exchange=ASX/trade_...,e8f5dbbd135334b35c5c46fb26bb5791154eb4628a73d8...
